# Functions

## Defining Functions

`return` doesn't required. Any function returns `None` by default:

In [1]:
def foo():
    42

In [2]:
print(foo())

None


### Documentation

We can use string literals for documenting the function.

In [3]:
def foo():
    """Returns some magic number"""
    return 42

In [4]:
foo.__doc__

'Returns some magic number'

In [5]:
help(foo) # or foo? in IPython

Help on function foo in module __main__:

foo()
    Returns some magic number



---

In [6]:
def min(x, y):
    return x if x < y else y

In [7]:
min(-5, 12)

-5

## Keyword Arguments

In [8]:
min(x=-5, y=12)

-5

In [9]:
min(x=-5, z=12)

TypeError: min() got an unexpected keyword argument 'z'

In [10]:
min(y=12, x=-5)

-5

### Mutable Default Arguments

In [36]:
def unique(iterable, seen=set()):
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc

In [37]:
xs = [1, 1, 2, 3]
unique(xs)

[1, 2, 3]

In [38]:
unique(xs)

[]

In [39]:
unique.__defaults__

({1, 2, 3},)

How we can avoid this gotcha?

In [41]:
def unique(iterable, seen=None):
    seen = set(seen or [])
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc

In [42]:
xs = [1, 1, 2, 3]
unique(xs)

[1, 2, 3]

In [43]:
unique(xs)

[1, 2, 3]

In [44]:
unique.__defaults__

(None,)

### Required Keyword Arguments

In [45]:
def flatten(xs, depth=None):
    pass

In [46]:
flatten([1, [2], 3], depth=1)
flatten([1, [2], 3], 1)

In [47]:
# if you want to require keyword arguments in function calls
def flatten(xs, *, depth=None):
    pass

In [48]:
flatten([1, [2], 3], 1)

TypeError: flatten() takes 1 positional argument but 2 were given

In [50]:
# More explicit function call
flatten([1, [2], 3], depth=1)

## Packing and Unpacking Arguments

In [16]:
def min(*args):  # type(args) == tuple
    result = float('inf')
    for arg in args:
        if arg < result:
            result = arg
    return result

ℹ️ _Python idiom: `float('inf')`_. It acts as an unbounded upper value for comparison.

In [14]:
min(-5, 12, 13)

-5

In [15]:
min()

inf

Function with at least one required argument:

In [62]:
def min(first, *args):
    result = first
    for arg in args:
        if arg < result:
            result = arg
    return result

In [20]:
min()

TypeError: min() missing 1 required positional argument: 'first'

💡 _Unpacking works with any object implementing iterator protocol._

In [27]:
# set
# only for educational purposes, avoid unpacking sets – they're unordered!
xs = {-5, 12, 13}
min(*xs)

-5

In [25]:
# list
min(*[-5, 12, 13])

-5

In [26]:
# tuple
min(*(-5, 12, 13))

-5

⚠️ _Using unpacking, you always have to remember the order of function arguments._

---

In [30]:
def bounded_min(first, *args, lo=float('-inf'), hi=float('inf')):
    result = hi
    for arg in (first,) + args:
        if arg < result and lo < arg < hi:
            result = arg
    return max(result, lo)

In [33]:
bounded_min(-5, 12, 13)

-5

In [31]:
bounded_min(-5, 12, 13, lo=0, hi=255)

12

In [35]:
bounded_min(-5, 12, 13, lo=100, hi=200)

200

ℹ️ _Python idiom: `lo < arg < hi`._ It simplifies the condition.

### Keyword Arguments

In [55]:
def runner(cmd, **kwargs):
    if kwargs.get('verbose', True):
        print('Logging enabled')
    else:
        print('Logging disabled')

In [56]:
runner('dramatiq', processes=4, threads=4)

Logging enabled


In [57]:
runner('dramatiq', processes=4, threads=4, verbose=False)

Logging disabled


In [59]:
runner('dramatiq', **{'processes': 2, 'threads': 2, 'verbose': False})

Logging disabled


In [60]:
options = {'processes': 16, 'threads': 16, 'verbose': False}
runner('dramatiq', **options)

Logging disabled


### Unpacking and Assignment

In [63]:
acc = []
seen = set()

In [64]:
# using unpacking with assignment
(acc, seen) = ([], set())

In [69]:
# We can un
x, y, z = [1, 2, 3]
x, y, z = {1, 2, 3} # unordered!
x, y, z = "123"

In [70]:
x

'1'

In [71]:
y

'2'

In [72]:
z

'3'

Parentheses are not required, but sometimes they can be useful:

In [74]:
rectangle = (0, 0), (4, 4)
(x1, y1), (x2, y2) = rectangle

### Extended Iterable Unpacking

[PEP-3132](https://www.python.org/dev/peps/pep-3132/)

In [75]:
first, *rest = range(1, 5)

In [76]:
first, rest

(1, [2, 3, 4])

In [77]:
first, *rest, last = range(1, 5)
last

4

In [78]:
rest

[2, 3]

In [79]:
first, *rest, last = [42]

ValueError: not enough values to unpack (expected at least 2, got 1)

---

In [80]:
import dis
dis.dis("first, *rest, last = ('a', 'b', 'c')")

  1           0 LOAD_CONST               0 (('a', 'b', 'c'))
              2 EXTENDED_ARG             1
              4 UNPACK_EX              257
              6 STORE_NAME               0 (first)
              8 STORE_NAME               1 (rest)
             10 STORE_NAME               2 (last)
             12 LOAD_CONST               1 (None)
             14 RETURN_VALUE


As we see, variable assignment works from left to right.

In [81]:
x, (x, y) = 1, (2, 3)

In [82]:
x

2

`x` is equal 2 as expected.

P.S. We used variable `x` twice in the expression just for educational purposes. This could be confusing for others.

In [83]:
dis.dis("first, *rest, last = ['a', 'b', 'c']")

  1           0 LOAD_CONST               0 ('a')
              2 LOAD_CONST               1 ('b')
              4 LOAD_CONST               2 ('c')
              6 BUILD_LIST               3
              8 EXTENDED_ARG             1
             10 UNPACK_EX              257
             12 STORE_NAME               0 (first)
             14 STORE_NAME               1 (rest)
             16 STORE_NAME               2 (last)
             18 LOAD_CONST               3 (None)
             20 RETURN_VALUE


💡 _Unpacking the list works differently on runtime._

---

In [84]:
def f(*args, **kwargs):
    pass

In [85]:
f(1, 2, 3, **{'foo': 42})

In [86]:
first, *rest = range(4)

In [87]:
for first, *rest in [range(4), range(2)]:
    pass

#### Python 3.5 – Additional Unpacking Generalizations

[PEP-0448](https://www.python.org/dev/peps/pep-0448/)

In [88]:
def f(*args, **kwargs):
    print(args, kwargs)

In [89]:
f(1, 2, *[3, 4], *[5], foo='bar', **{'baz': 42}, boo=24)

(1, 2, 3, 4, 5) {'foo': 'bar', 'baz': 42, 'boo': 24}


In [90]:
defaults = {'host': '0.0.0.0', 'port': 8080}

In [91]:
{**defaults, 'port': 80}

{'host': '0.0.0.0', 'port': 80}

In [92]:
[*range(5), 6]

[0, 1, 2, 3, 4, 6]

## Function Scope

Functions are first-class citizens in Python.

In [93]:
def wrapper():
    def identity(x):
        return x
    return identity

In [95]:
f = wrapper()
f(42)

42

In [96]:
def make_min(*, lo, hi):
    """Returns configured min function"""
    def inner(first, *args):
        result = hi
        for arg in (first, ) + args:
            if arg < result and lo < arg < hi:
                result = arg
        return max(result, lo)
    return inner

In [97]:
bounded_min = make_min(lo=0, hi=255)
bounded_min(-5, 12, 13)

12

[Python scoping: understanding LEGB](https://blog.mozilla.org/webdev/2011/01/31/python-scoping-understanding-legb/)

In [100]:
min      # builtin
min = 42 # global

def f(*args):
    min = 2
    def g():    # enclosing
        min = 4 # local
        print(min)

---

In [103]:
min = 42  # globals()['min'] = 42
globals() # globals is mutable dict

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'def foo():\n    42',
  'print(foo())',
  'def foo():\n    """Returns some magic number"""\n    return 42',
  'foo.__doc__',
  'help(foo)',
  'def min(x, y):\n    return x if x < y else y',
  'min(-5, 12)',
  'min(x=-5, y=12)',
  'min(x=-5, z=12)',
  'min(y=12, x=-5)',
  'def min(*args): # type(args) == tuple\n    result = None\n    for arg in args:\n        if arg < result:\n            result = arg\n    return result',
  'min(-5, 12, 13)',
  "def min(*args): # type(args) == tuple\n    result = float('inf')\n    for arg in args:\n        if arg < result:\n            result = arg\n    return result",
  'min(-5, 12, 13)',
  'min()',
  "def min(*args):  # type(args) == tuple\n    result = float('inf')  # remember

In [104]:
def f():
    min = 2 # locals['min'] = 2
    print(locals())
    

f()

{'min': 2}


ℹ️ Function can use variables defined outside of function scope.

⚠️ _Lookup of variables happens during runtime:_

In [106]:
def f():
    print(i)

In [107]:
for i in range(4):
    f()

0
1
2
3


⚠️ The LEGB rule doesn't work for assignment:

In [110]:
min = 42
def f():
    min += 1   # <- lookup in a global scope!
    return min

In [111]:
f()

UnboundLocalError: local variable 'min' referenced before assignment

This behavior can be changed by using `global` and `nonlocal` operators.

In [117]:
min = 42
def f():
    global min
    min += 1   # <- lookup in a global scope!
    return min

In [118]:
f()
f()

44

⚠️ _Try to avoid global variables._

[PEP 3104 -- Access to Names in Outer Scopes](https://www.python.org/dev/peps/pep-3104/)

In [119]:
def cell(value=None):
    def get():
        return value
    def set(update):
        nonlocal value
        value = update
    return get, set

In [120]:
get, set = cell()

In [121]:
set(42)

In [122]:
get()

42

## Elements Of Functional Programming

### Anonymous functions

In [128]:
lambda name: print('Hi ' + name + '!')

<function __main__.<lambda>(name)>

In [130]:
lambda name, *args, age=30, **kwargs: 42

<function __main__.<lambda>(name, *args, age=30, **kwargs)>

### Map, Filter and Zip

In [132]:
def identity(x):
    return x

In [133]:
map(identity, range(4))

In [134]:
list(map(identity, range(4)))

[0, 1, 2, 3]

In [141]:
set(map(lambda x: x % 7, [1, 9, 16, -1, 2, 5]))

In [142]:
list(map(lambda x, n: x ** n, [2, 3], range(1, 8)))

[2, 9]

---

In [143]:
filter(lambda x: x % 2 != 0, range(10))

In [144]:
list(filter(lambda x: x % 2 != 0, range(10)))

[1, 3, 5, 7, 9]

In [147]:
xs = [0, None, [], {}, '', 42]
# returns truthy values
list(filter(None, xs))

[42]

---

In [148]:
list(zip('abc', range(3), [42j, 42j, 42j]))

[('a', 0, 42j), ('b', 1, 42j), ('c', 2, 42j)]

In [149]:
list(zip('abc', range(10)))

[('a', 0), ('b', 1), ('c', 2)]

### Comprehensions

Come from ABC language. It's good alternative to `map` and `filter`.

#### Lists

In [150]:
[x ** 2 for x in range(10) if x % 2 == 1]

[1, 9, 25, 49, 81]

In [152]:
list(map(lambda x: x ** 2,
        filter(lambda x: x % 2 == 1,
              range(10))))

[1, 9, 25, 49, 81]

In [153]:
nested = [range(5), range(8, 10)]
[x for xs in nested for x in xs]  # flatten

[0, 1, 2, 3, 4, 8, 9]

⚠️ _Nested comprehensions are hard to understand._

#### Sets and Dicts

In [156]:
{x % 7 for x in [1, 9, 16, -1, 2, 5]}

{1, 2, 5, 6}

In [157]:
date = {'year': 2020, 'month': 'March', 'day': ''}
{k: v for k, v in date.items() if v}

{'year': 2020, 'month': 'March'}

In [158]:
{x: x**2 for x in range(4)}

{0: 0, 1: 1, 2: 4, 3: 9}